In [14]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import optuna
import logging
import sys

from typing import Any

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


# Load data

In [15]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# NN Model

In [16]:
class NeuralNetwork(nn.Module):
    def __init__(self, hidden_layers_types: list[str], hidden_layers_outputs: list[int]):
        super(NeuralNetwork, self).__init__()

        inputs = 28*28
        outputs = 10
        layers = []

        for layer_type, layer_outputs in zip(hidden_layers_types, hidden_layers_outputs):
            layers.append(nn.Linear(inputs, layer_outputs))
            if layer_type == 'relu':
                layers.append(nn.ReLU())
            elif layer_type == 'tanh':
                layers.append(nn.Tanh())
            elif layer_type == 'sigmoid':
                layers.append(nn.Sigmoid())
            inputs = layer_outputs

        layers.append(nn.Linear(inputs, outputs))
        layers.append(nn.Softmax())

        self.flatten = nn.Flatten()
        self.layers_sequence = nn.Sequential(*layers)

    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers_sequence(x)
        return logits

# Training

In [17]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    # print(f"test accuracy: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f}")
    return correct * 100

# Optuna

In [19]:
def objective(trial):

    epochs = 10

    num_layers = trial.suggest_int("num_layers", 1, 3)

    layer_1 = trial.suggest_categorical("layer_1", ["relu", "tanh", "sigmoid"])
    layer_2 = trial.suggest_categorical("layer_2", ["relu", "tanh", "sigmoid"])
    layer_3 = trial.suggest_categorical("layer_3", ["relu", "tanh", "sigmoid"])

    units_1 = trial.suggest_int("units_1", 300, 600, step=25)
    units_2 = trial.suggest_int("units_2", 150, 500, step=25)
    units_3 = trial.suggest_int("units_3", 75, 300, step=25)

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_int("batch_size", 32, 512, log=True)

    layers = [layer_1, layer_2, layer_3]
    units = [units_1, units_2, units_3]
    model = NeuralNetwork(layers[:num_layers], units[:num_layers]).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    train_dataloader = DataLoader(training_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    best = 0
    for _ in range(epochs):
        train(train_dataloader, model, loss_fn, optimizer)
        score = test(test_dataloader, model, loss_fn)
        best = max(best, score)
    return best


# Test

In [20]:
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=12)

pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

trial = study.best_trial

print("Best trial:")
print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-06-22 20:48:42,719] A new study created in memory with name: no-name-eba49923-6e99-4cac-8437-a452f57a00f8


A new study created in memory with name: no-name-eba49923-6e99-4cac-8437-a452f57a00f8
A new study created in memory with name: no-name-eba49923-6e99-4cac-8437-a452f57a00f8


[W 2022-06-22 20:48:42,734] Trial 0 failed because of the following error: TypeError("float() argument must be a string or a real number, not 'tuple'")
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\py310\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Piotr Biały\AppData\Local\Temp\ipykernel_632\2616892374.py", line 28, in objective
    best = TrialResult((0, None))
TypeError: float() argument must be a string or a real number, not 'tuple'


Trial 0 failed because of the following error: TypeError("float() argument must be a string or a real number, not 'tuple'")
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\py310\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Piotr Biały\AppData\Local\Temp\ipykernel_632\2616892374.py", line 28, in objective
    best = TrialResult((0, None))
TypeError: float() argument must be a string or a real number, not 'tuple'
Trial 0 failed because of the following error: TypeError("float() argument must be a string or a real number, not 'tuple'")
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\py310\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Piotr Biały\AppData\Local\Temp\ipykernel_632\2616892374.py", line 28, in objective
    best = TrialResult((0, None))
TypeError: float() argument must be a strin

TypeError: float() argument must be a string or a real number, not 'tuple'